In [1]:
import ssl
# Bypass SSL verification
ssl._create_default_https_context = ssl._create_unverified_context
import os
os.environ['CURL_CA_BUNDLE'] = ''

import weaviate
import numpy as np
from sentence_transformers import SentenceTransformer

# Connect to Weaviate
client = weaviate.Client("http://weaviate:8080")

# Load Sentence Transformer Model for Embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Function to insert a question
def insert_question(question_text):
    embedding = model.encode([question_text]).tolist()[0]
    question_object = {
        "text": question_text,
        "embedding": embedding
    }
    client.data_object.create(question_object, "Question")

# Function to query for similar questions
def fetch_similar_questions(question_text, threshold=0.8):
    input_embedding = model.encode([question_text])
    
    query = client.query.get("Question", ["text", "embedding"]).do()
    questions = query['data']['Get']['Question']
    
    similar_questions = []
    for question in questions:
        question_embedding = np.array(question['embedding']).reshape(1, -1)
        similarity = cosine_similarity(input_embedding, question_embedding)[0][0]
        if similarity >= threshold:
            similar_questions.append(question['text'])
    
    return similar_questions

# Insert and fetch questions
insert_question("What is your favorite product?")
print(fetch_similar_questions("What's your favorite product and why?"))


/opt/conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/tmp/ipykernel_633/3936958711.py:12: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client("http://weaviate:

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))"), '(Request ID: 8505bb84-6342-415e-b0e2-a8239aae7553)')